In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import numpy as np 

Using TensorFlow backend.


In [2]:
tokenizer = Tokenizer()

def dataset_preparation(data):

    # basic cleanup
    corpus = data.lower().split("\n")

    # tokenization    
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

In [4]:
cut_size = 10000
data = open('datasets/DomCasmurro.txt').read()[:cut_size]
print(data)

DOM CASMURRO

POR

MACHADO DE ASSIS

DA ACADEMIA BRAZILEIRA

H. GARNIER, LIVREIRO-EDITOR

RUA MOREIRA CEZAR, 71

RIO DE JANEIRO

6, RUE DES SAINTS-PÈRES, 6

PARIZ




I

Do titulo.

Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no
trem da Central um rapaz aqui do bairro, que eu conheço de vista e
de chapéo. Comprimentou-me, sentou-se ao pé de mim, falou da lua e
dos ministros, e acabou recitando-me versos. A viagem era curta, e os
versos póde ser que não fossem inteiramente maus. Succedeu, porém, que
como eu estava cançado, fechei os olhos tres ou quatro vezes; tanto
bastou para que elle interrompesse a leitura e mettesse os versos no
bolso.

--Continue, disse eu accordando.

--Já acabei, murmurou elle.

--São muito bonitos.

Vi-lhe fazer um gesto para tiral-os outra vez do bolso, mas não passou
do gesto; estava amuado. No dia seguinte entrou a dizer de mim nomes
feios, e acabou alcunhando-me _Dom Casmurro._ Os visinhos, que não
gostam dos meus habitos reclusos e cal

In [5]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)

In [6]:
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences = True))
#model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 10)            7770      
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 150)           96600     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               100400    
_________________________________________________________________
dense_1 (Dense)              (None, 777)               78477     
Total params: 283,247
Trainable params: 283,247
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model.fit(predictors, label, epochs=100, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1602/1602 [==============================] - 2s 1ms/step - loss: 6.3881 - acc: 0.0406
Epoch 2/100
1602/1602 [==============================] - 1s 572us/step - loss: 5.9572 - acc: 0.0456
Epoch 3/100
1602/1602 [==============================] - 1s 554us/step - loss: 5.8920 - acc: 0.0456
Epoch 4/100
1602/1602 [==============================] - 1s 572us/step - loss: 5.8751 - acc: 0.0456
Epoch 5/100
1602/1602 [==============================] - 1s 564us/step - loss: 5.8658 - acc: 0.0437
Epoch 6/100
1602/1602 [==============================] - 1s 577us/step - loss: 5.8592 - acc: 0.0456
Epoch 7/100
1602/1602 [==============================] - 1s 579us/step - loss: 5.8560 - acc: 0.0456
Epoch 8/100
1602/1602 [==============================] - 1s 565us/step - loss: 5.8428 - acc: 0.0362
Epoch 9/100
1602/1602 [==============================] - 1s 567us/step - loss: 5.8121 - acc: 0.0456
Epoch 10/100
1602/1602 [==============================

In [8]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [9]:
seed_text = "Você o que quer é"
next_words = 20
print(generate_text(seed_text, next_words, max_sequence_len))

Você o que quer é é lhe aspecto e acabou dias tenho visto os pequenos rua e mettido falo outras de menos guardo pegasse da
